Номер группы, ФИО: 402Б Билецкий Арсений Олегович

In [111]:
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

Загрузить данные load_wine. (параметр return_X_y=True позволит вернуть данные в виде объекта tuple X, y). Разбить данные на тренировочные и проверочные в соотношении 7/3. Зафиксировать random_state=5.

In [112]:
wine = load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(wine[0], wine[1], test_size=0.3, random_state=5)

## Сравнение результатов работы алгоритмов после применения MinMaxScaler и StandardScaler

In [113]:
np.set_printoptions(suppress=True)#не отображать числа с плавающей точкой как e+00, 
#отображать как 0.0012345

* Сделать массив классификаторов `[Perceptron, LogisticRegression, KNeighborsClassifier(n_neighbors=7), DecisionTreeClassifier(max_depth=3)]` 

* Создать масштабирующие классы `StandatdScaler` и `MinMaxScaler`. 

* Отмасштабировать тренировочные (`fit_transform()`) и тестовые (`transform()`) признаки и тем и другим классом.

* Поочередно оценить точность прогнозов на немсаштабированных данных, данных после `MinMaxScaler`, после `StandardScaler` для алгоритмов из массива.

* Отобразить точность для каждого случая (название класса можно вывести как `clfr.__class__.__name__`)

In [124]:
clr = [Perceptron(max_iter=1000, tol=1e-3), 
          LogisticRegression(multi_class='auto',solver='liblinear'),
          KNeighborsClassifier(n_neighbors=7), 
          DecisionTreeClassifier(max_depth=3)]
clr[0].__class__.__name__

'Perceptron'

In [125]:
def Fit_Predict(X,y,X_test):
    A=[]
    N=[]
    for i in range(len(clr)):
        clr[i].fit(X, y) 
        A.append(clr[i].predict(X_test))
        N.append(clr[i].__class__.__name__)
    return A,N

In [126]:
scaler = StandardScaler()
scaled = scaler.fit_transform(X_train)
test_scaled=scaler.transform(X_test)

In [127]:
minmaxscaler = MinMaxScaler()
minmaxscaled = minmaxscaler.fit_transform(X_train)
test_minmaxscaled=minmaxscaler.transform(X_test)

In [128]:
Predict, Name_1=Fit_Predict(X_train, y_train, X_test)
Predict_scaler, Name_2=Fit_Predict(scaled, y_train, test_scaled)
Predict_minmaxscaler, Name_3=Fit_Predict(minmaxscaled, y_train, test_minmaxscaled)

In [129]:
print('Прогноз на немсаштабированных данных')
for i in range(len(clr)):
    print(Name_1[i],round(metrics.accuracy_score(y_test, Predict[i]),3))

Прогноз на немсаштабированных данных
Perceptron 0.593
LogisticRegression 0.926
KNeighborsClassifier 0.759
DecisionTreeClassifier 0.926


In [130]:
print('Прогноз на данных после StandardScaler()')
for i in range(len(clr)):
    print(Name_2[i],round(metrics.accuracy_score(y_test, Predict_scaler[i]),3))

Прогноз на данных после StandardScaler()
Perceptron 1.0
LogisticRegression 1.0
KNeighborsClassifier 0.981
DecisionTreeClassifier 0.926


In [131]:
print('Прогноз на данных после MinMaxScaler()')
for i in range(len(clr)):
    print(Name_3[i],round(metrics.accuracy_score(y_test, Predict_minmaxscaler[i]),3))

Прогноз на данных после MinMaxScaler()
Perceptron 0.926
LogisticRegression 0.944
KNeighborsClassifier 0.963
DecisionTreeClassifier 0.926


Ответить на вопросы:
1. В каком случае (StandatdScaler/MinMaxScaler/без масштабирования) алгоритмы показали лучший результат?

Ответ: <font color='green'>Лучший результат показали классификаторы Perceptron, LogisticRegression с использованием StandardScaler()</font>

2. Как повлияло масштабирование признаков на результаты работы DecisionTreeClassifier?

Ответ: <font color='green'>Никак</font>

Для классификатора `LogisticRegression` сделать конвейер с лучшим алгоритмом масштабирования, полученным в предыдущем задании. Обучить конвейер на тренировочных данных, сделать предсказания с помощью конвейера, оценить точность на тестовых данных.
Из конвейера получить натренированный классификатор (`pipline.steps[0]` вернет `tuple` с масштабирующим классом, `pipline.steps[1]` вернет `tuple` с классификатором, откуда можно получить классификатор)

In [142]:
from sklearn.pipeline import Pipeline
pipe = Pipeline(steps=[('scaler', StandardScaler()),('clf', LogisticRegression(multi_class='auto',solver='liblinear'))])
pipe.fit(X_train, y_train)
round(metrics.accuracy_score(y_test, pipe.predict(X_test)),3)

1.0

In [134]:
pipe.steps[0]

('scaler', StandardScaler())

In [135]:
pipe.steps[1]

('clf', LogisticRegression(solver='liblinear'))

Отобразить коэффициенты весов `w` разделяющих классы гиперплоскостей (3 массива весов, которые отделяют каждый класс от двух других)

In [147]:
w=np.array(pipe.named_steps['clf'].coef_)
w

array([[ 1.19492626,  0.13597831,  0.47455935, -1.27276261, -0.00591202,
         0.31594293,  0.92846648, -0.26177061, -0.0957287 ,  0.13128945,
         0.06106702,  0.64448827,  1.72510196],
       [-1.39030608, -0.50592331, -0.80222654,  0.64718261, -0.04381452,
        -0.10982866,  0.49686606,  0.29667133,  0.21979153, -1.89590739,
         0.84135887,  0.08353773, -1.65794101],
       [ 0.33611661,  0.35644895,  0.32539937,  0.32404121,  0.07326638,
        -0.01396881, -1.31261802, -0.03135595, -0.33526387,  1.65677635,
        -0.83741372, -0.79692207, -0.13077839]])

In [145]:
names = np.array(['Алкоголь','Яблочная кислота','Зола','Щелочность золы','Магнезия',
                  'Общее содержание фенолов', 'Флаваноиды', 'Нефлаваноидные фенолы', 
                  'Проантоцианины', 'Интенсивность цвета', 'Оттенок', 
                  'OD280 / OD315 разбавленных (разведенных) вин', 'Пролин'])

По коэффициентам весов `w` классификатора  определить, какие из признаков наиболее значимы (имеют абсолютное значение больше единицы). <b>Важно:</b> для отбора признаков использовать возможности библиотеки NumPy (см. lection_2_numpy_start)

In [162]:
w[0]>abs(1)

array([ True, False, False, False, False, False, False, False, False,
       False, False, False,  True])

In [158]:
import_signs=[]
for i in range(len(w[0])):
    if w[0][i]>abs(1) or w[1][i]>abs(1) or w[2][i]>abs(1):
        print(names[i])

Алкоголь
Интенсивность цвета
Пролин
